In [1]:
"""
调参技巧
"""
# 可视化
# fine-tune
# 优化算法、激活函数、初始化函数
# 批归一化
# 数据增强

'\n调参技巧\n'

In [2]:
# tensorboard
# 1. 指定面板图上显示的变量
# 2. 训练过程中将这些变量计算出来，输出到文件中
# 3. 文件解析 ./tensorboard --logdir=dir.

In [3]:
import tensorflow as tf
import numpy as np
import pickle
import os

CIFAR_DIR = './../cifar-10-batches-py'
print(os.listdir(CIFAR_DIR))

['batches.meta', 'data_batch_1', 'data_batch_2', 'data_batch_3', 'data_batch_4', 'data_batch_5', 'readme.html', 'test_batch']


In [4]:
def load_data(filename):
    with open(filename, 'rb') as f:
        data = pickle.load(f, encoding='bytes')
        return data[b'data'], data[b'labels']

class CifarData:
    def __init__(self, filenames, need_shuffle):
        all_data = []
        all_labels = []
        for filename in filenames:
            data, labels = load_data(filename)
            all_data.append(data)
            all_labels.append(labels)
        self._data = np.vstack(all_data)
        self._data = self._data / 127.5 - 1  # 为了提高预测准确率，需要对数据进行归一化
        self._labels = np.hstack(all_labels)
        print(self._data.shape)
        print(self._labels.shape)
        
        self._num_examples = self._data.shape[0]
        self._need_shuffle = need_shuffle
        self._indicator = 0  # 当前数据集指明遍历的位置
        if self._need_shuffle:
            self._shuffle_data()
    
    def _shuffle_data(self):
        p = np.random.permutation(self._num_examples)  # 索引进行混排
        self._data = self._data[p]
        self._labels = self._labels[p]
    
    def next_batch(self, batch_size):
        """返回batch_size个样本"""
        end_indicator = self._indicator + batch_size
        if end_indicator > self._num_examples:
            if self._need_shuffle:
                self._shuffle_data()
                self._indicator = 0
                end_indicator = batch_size
            else:
                raise Exception("have no more examples")
        if end_indicator > self._num_examples:
            raise Exception("batch size is larger than all examples")
        batch_data = self._data[self._indicator: end_indicator]
        batch_labels = self._labels[self._indicator: end_indicator]
        self._indicator = end_indicator
        return batch_data, batch_labels

train_filenames = [os.path.join(CIFAR_DIR, 'data_batch_%d' % i) for i in range(1, 6)]
test_filenames = [os.path.join(CIFAR_DIR, 'test_batch')]

train_data = CifarData(train_filenames, True)
test_data = CifarData(test_filenames, False)

(50000, 3072)
(50000,)
(10000, 3072)
(10000,)


In [5]:
# 计算图模型，先新建计算图，然后将数据塞入计算图中
# placeholder 占位符的作用
x = tf.placeholder(tf.float32, [None, 3072])  # None 样本数不确定，mini_batch
# [None]
y = tf.placeholder(tf.int64, [None])

# 图片转换，reshape
x_image = tf.reshape(x, [-1, 3, 32, 32])
# 大小： 32*32
x_image = tf.transpose(x_image, perm=[0, 2, 3, 1])

# 第一个卷积层
# 参数说明
# - 32: 卷积层输出的通道数目
# - (3, 3): 卷积核的大小
# - padding: 类型有两种 same|valid，same的话使得输出的图像大小不会变化
# - activation: 激活函数
# - name: 给这一层做一个命名，将图打印出来有意义的的一个图
# conv1: 神经元图，feature_map, 输出图像
conv1_1 = tf.layers.conv2d(x_image, 32, (3, 3), padding='same', activation=tf.nn.relu, name='conv1_1')

conv1_2 = tf.layers.conv2d(conv1_1, 32, (3, 3), padding='same', activation=tf.nn.relu, name='conv1_2')

# pooling 池化层
# 参数说明
# - conv1: 输入参数
# - (2, 2): kernal size
# - (2, 3): 步长 (stride)
# - name: 给这一层做一个命名，将图打印出来有意义的的一个图
# 大小： 16*16
pooling1 = tf.layers.max_pooling2d(conv1_2, (2, 2), (2, 2), name='pool1')

conv2_1 = tf.layers.conv2d(pooling1, 32, (3, 3), padding='same', activation=tf.nn.relu, name='conv2_1')

conv2_2 = tf.layers.conv2d(conv2_1, 32, (3, 3), padding='same', activation=tf.nn.relu, name='conv2_2')
# 大小： 8*8
pooling2 = tf.layers.max_pooling2d(conv2_2, (2, 2), (2, 2), name='pool2')

conv3_1 = tf.layers.conv2d(pooling2, 32, (3, 3), padding='same', activation=tf.nn.relu, name='conv3_1')
conv3_2 = tf.layers.conv2d(conv3_1, 32, (3, 3), padding='same', activation=tf.nn.relu, name='conv3_2')
# 大小： 4*4*32
pooling3 = tf.layers.max_pooling2d(conv3_2, (2, 2), (2, 2), name='pool3')

# 做一个展平
# 展平后的shape: [None, 4*4*32]
flatten = tf.layers.flatten(pooling3)

# 全连接层，使它能够映射到10个类别上去
y_ = tf.layers.dense(flatten, 10)

# 交叉熵损失数函数
loss = tf.losses.sparse_softmax_cross_entropy(labels=y, logits=y_)
# 执行原理
# y_ -> softmax
# y -> one_hot
# loss -> ylogy_

# 寻找最大概率值的 indices
# 1 代表在第一个维度上
predict = tf.argmax(y_, 1)
# correct_prediction 结果可能是[0,1,0,0,1,1,1]
correct_prediction = tf.equal(predict, y)
# 求准确率，就是求平均值
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float64))

# 在我们有了目标函数和损失函数，再定义梯度下降方法
# 梯度下降的变种
# 优化最小loss的值
with tf.name_scope('train_op'):
    train_op = tf.train.AdamOptimizer(1e-3).minimize(loss)
# 计算图构建完成

Instructions for updating:
Use `tf.keras.layers.Conv2D` instead.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Use keras.layers.MaxPooling2D instead.


Instructions for updating:
Use keras.layers.flatten instead.
Instructions for updating:
Use keras.layers.dense instead.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


In [6]:
# 定义 init函数，执行初始化
init = tf.global_variables_initializer()
batch_size = 20
train_steps = 100000
test_steps = 100
# session，打开session后执行计算图
with tf.Session() as sess:
    sess.run(init)
    for i in range(train_steps):
        batch_data, batch_labels = train_data.next_batch(batch_size)
        loss_val, acc_val, _ = sess.run([loss, accuracy, train_op], feed_dict={x: batch_data, y: batch_labels})
        if (i+1) % 100 == 0:
            print('[Train] Step: %d, loss: %4.5f, acc: %4.5f' % (i+1, loss_val, acc_val))
        if (i+1) % 1000 == 0:
            test_data = CifarData(test_filenames, False)
            all_test_acc_val = []
            for j in range(test_steps):
                test_batch_data, test_batch_labels = test_data.next_batch(batch_size)
                test_acc_val = sess.run([accuracy], feed_dict={x: test_batch_data, y: test_batch_labels})
                all_test_acc_val.append(test_acc_val)
            test_acc = np.mean(all_test_acc_val)
            print('[Test ] Step: %d, acc: %4.5f' % (i+1, test_acc))

[Train] Step: 100, loss: 1.85634, acc: 0.35000
[Train] Step: 200, loss: 1.86575, acc: 0.45000
[Train] Step: 300, loss: 1.78467, acc: 0.40000
[Train] Step: 400, loss: 1.87614, acc: 0.30000
[Train] Step: 500, loss: 1.91084, acc: 0.25000
[Train] Step: 600, loss: 1.55235, acc: 0.25000
[Train] Step: 700, loss: 1.63955, acc: 0.40000
[Train] Step: 800, loss: 1.65670, acc: 0.40000
[Train] Step: 900, loss: 1.60237, acc: 0.45000
[Train] Step: 1000, loss: 1.51815, acc: 0.55000
(10000, 3072)
(10000,)
[Test ] Step: 1000, acc: 0.51750
[Train] Step: 1100, loss: 1.28620, acc: 0.50000
[Train] Step: 1200, loss: 1.13470, acc: 0.50000
[Train] Step: 1300, loss: 1.19133, acc: 0.65000
[Train] Step: 1400, loss: 1.50159, acc: 0.40000
[Train] Step: 1500, loss: 1.64293, acc: 0.45000
[Train] Step: 1600, loss: 1.70861, acc: 0.35000
[Train] Step: 1700, loss: 1.32741, acc: 0.45000
[Train] Step: 1800, loss: 1.04880, acc: 0.70000
[Train] Step: 1900, loss: 1.01060, acc: 0.65000
[Train] Step: 2000, loss: 1.13868, acc: 0

[Train] Step: 15400, loss: 0.36394, acc: 0.90000
[Train] Step: 15500, loss: 0.38761, acc: 0.85000
[Train] Step: 15600, loss: 0.71818, acc: 0.75000
[Train] Step: 15700, loss: 1.29245, acc: 0.60000
[Train] Step: 15800, loss: 0.75236, acc: 0.75000
[Train] Step: 15900, loss: 0.84840, acc: 0.55000
[Train] Step: 16000, loss: 0.41771, acc: 0.85000
(10000, 3072)
(10000,)
[Test ] Step: 16000, acc: 0.74000
[Train] Step: 16100, loss: 0.30598, acc: 0.90000
[Train] Step: 16200, loss: 0.59433, acc: 0.75000
[Train] Step: 16300, loss: 0.84176, acc: 0.65000
[Train] Step: 16400, loss: 0.56957, acc: 0.75000
[Train] Step: 16500, loss: 0.58433, acc: 0.75000
[Train] Step: 16600, loss: 1.06285, acc: 0.65000
[Train] Step: 16700, loss: 0.66252, acc: 0.75000
[Train] Step: 16800, loss: 0.29141, acc: 1.00000
[Train] Step: 16900, loss: 0.30731, acc: 0.85000
[Train] Step: 17000, loss: 0.72640, acc: 0.75000
(10000, 3072)
(10000,)
[Test ] Step: 17000, acc: 0.73450
[Train] Step: 17100, loss: 0.37430, acc: 0.90000
[Tra

[Train] Step: 30400, loss: 0.55609, acc: 0.85000
[Train] Step: 30500, loss: 1.28615, acc: 0.75000
[Train] Step: 30600, loss: 0.54507, acc: 0.80000
[Train] Step: 30700, loss: 0.88949, acc: 0.70000
[Train] Step: 30800, loss: 0.30266, acc: 0.90000
[Train] Step: 30900, loss: 0.46026, acc: 0.80000
[Train] Step: 31000, loss: 0.55032, acc: 0.80000
(10000, 3072)
(10000,)
[Test ] Step: 31000, acc: 0.74750
[Train] Step: 31100, loss: 0.63243, acc: 0.80000
[Train] Step: 31200, loss: 0.17114, acc: 1.00000
[Train] Step: 31300, loss: 0.44045, acc: 0.85000
[Train] Step: 31400, loss: 0.49304, acc: 0.75000
[Train] Step: 31500, loss: 0.68125, acc: 0.75000
[Train] Step: 31600, loss: 1.17788, acc: 0.70000
[Train] Step: 31700, loss: 0.21684, acc: 0.95000
[Train] Step: 31800, loss: 0.46517, acc: 0.85000
[Train] Step: 31900, loss: 0.18906, acc: 0.90000
[Train] Step: 32000, loss: 0.65864, acc: 0.80000
(10000, 3072)
(10000,)
[Test ] Step: 32000, acc: 0.73650
[Train] Step: 32100, loss: 0.48578, acc: 0.75000
[Tra

[Train] Step: 45400, loss: 0.49654, acc: 0.85000
[Train] Step: 45500, loss: 0.21985, acc: 0.95000
[Train] Step: 45600, loss: 0.43808, acc: 0.75000
[Train] Step: 45700, loss: 0.38437, acc: 0.90000
[Train] Step: 45800, loss: 0.24140, acc: 0.90000
[Train] Step: 45900, loss: 0.63223, acc: 0.70000
[Train] Step: 46000, loss: 0.45916, acc: 0.85000
(10000, 3072)
(10000,)
[Test ] Step: 46000, acc: 0.73700
[Train] Step: 46100, loss: 0.53786, acc: 0.85000
[Train] Step: 46200, loss: 0.67005, acc: 0.85000
[Train] Step: 46300, loss: 0.34858, acc: 0.95000
[Train] Step: 46400, loss: 0.31351, acc: 0.95000
[Train] Step: 46500, loss: 0.08023, acc: 1.00000
[Train] Step: 46600, loss: 0.53299, acc: 0.85000
[Train] Step: 46700, loss: 0.34264, acc: 0.85000
[Train] Step: 46800, loss: 0.23806, acc: 0.90000
[Train] Step: 46900, loss: 0.29038, acc: 0.85000
[Train] Step: 47000, loss: 0.26991, acc: 0.85000
(10000, 3072)
(10000,)
[Test ] Step: 47000, acc: 0.74900
[Train] Step: 47100, loss: 0.28844, acc: 0.85000
[Tra

[Train] Step: 60400, loss: 1.31027, acc: 0.65000
[Train] Step: 60500, loss: 0.31935, acc: 0.85000
[Train] Step: 60600, loss: 0.77485, acc: 0.75000
[Train] Step: 60700, loss: 0.22015, acc: 0.90000
[Train] Step: 60800, loss: 0.89240, acc: 0.70000
[Train] Step: 60900, loss: 0.54738, acc: 0.85000
[Train] Step: 61000, loss: 0.50158, acc: 0.80000
(10000, 3072)
(10000,)
[Test ] Step: 61000, acc: 0.74200
[Train] Step: 61100, loss: 0.22657, acc: 0.85000
[Train] Step: 61200, loss: 0.30640, acc: 0.90000
[Train] Step: 61300, loss: 0.39827, acc: 0.90000
[Train] Step: 61400, loss: 0.21343, acc: 0.90000
[Train] Step: 61500, loss: 0.36912, acc: 0.85000
[Train] Step: 61600, loss: 0.32090, acc: 0.90000
[Train] Step: 61700, loss: 0.60515, acc: 0.75000
[Train] Step: 61800, loss: 0.18969, acc: 0.95000
[Train] Step: 61900, loss: 0.48199, acc: 0.85000
[Train] Step: 62000, loss: 0.27692, acc: 0.85000
(10000, 3072)
(10000,)
[Test ] Step: 62000, acc: 0.73000
[Train] Step: 62100, loss: 0.35132, acc: 0.85000
[Tra

KeyboardInterrupt: 